In [1]:
from typing import Generic, TypeVar, Callable

# function input T -> output S
T = TypeVar('T')  # value type T
S = TypeVar('S')  # value type S

E = TypeVar('E')  # error type E

# Maybe Monad

Maybe monad wraps a value with two possible states: `Just` and `Nothing`.


# Implementing `Maybe` Monad


In [2]:
class Maybe(Generic[T]):
    def __init__(self, value: T):
        self.value = value

    def is_nothing(self) -> bool: ...
    def is_just(self) -> bool: ...

    # haskell
    # bind :: m a -> (a -> m b) -> m b
    def bind(self, f: Callable[[T], 'Maybe[S]']) -> 'Maybe[S]': ...

    def __rshift__(self, f: Callable[[T], 'Maybe[S]']) -> 'Maybe[S]':
        # a >> func = a.bind(func)
        return self.bind(f)

In [3]:
class Just(Maybe[T]):
    """Wrapper for a value of type T"""

    def __init__(self, value: T):
        self.value = value

    def is_nothing(self) -> bool:
        return False

    def is_just(self) -> bool:
        return True

    def __repr__(self) -> str:
        return f'Just({self.value.__repr__()})'

    def bind(self, f: Callable[[T], Maybe[S]]) -> Maybe[S]:
        return f(self.value)

In [4]:
class Nothing(Maybe):
    """
    Wrapper for an error message (str)
    """

    def __init__(self, value: str):
        # use value to store error message
        self.value = value
        self.error = value

    def is_nothing(self) -> bool:
        return True

    def is_just(self) -> bool:
        return False

    def __repr__(self) -> str:
        return 'Nothing'

    def bind(self, f: Callable[[T], Maybe[S]]) -> Maybe[S]:
        # return self, which is Nothing(error_message)
        return self

# Divide function with `Maybe` Monad

```haskell
divide :: Float -> Float -> Maybe Float
divide x 0 = Nothing
divide x y = Just (x / y)
```


In [5]:
def divide(x: float, y: float) -> Maybe[float]:
    if y == 0:
        return Nothing('division by zero')
    return Just(x / y)


divided_by_two = lambda x: divide(x, 2)
divided_by_zero = lambda x: divide(x, 0)

In [6]:
divide(10, 0)

Nothing

In [7]:
divide(10, 2)

Just(5.0)

In [8]:
divided_by_two(10)

Just(5.0)

In [9]:
divided_by_zero(10)

Nothing

## But we cannot compose functions on values wrapped in Maybe monad

```haskell
wrapped_func :: a -> Maybe b
```


In [11]:
divided_by_two(divided_by_two(10))

TypeError: unsupported operand type(s) for /: 'Just' and 'int'

When chain calling we need a function that do something below

```haskell
bind :: m a -> (a -> m b) -> m b
```


# Take a break

Let's see the function like that we have used before `List.map`.

`map` type signature

```haskell
map :: (a -> b) -> [a] -> [b]
```

```python
# map :: (a -> b), [a]
# return [b]
map(func, iterable)
```


In [12]:
# arr :: [int]
arr = [1, 2, 3, 4, 5]

# divide_by_two :: int -> float
divide_by_two = lambda x: x / 2.0

# map :: (a -> b) -> [a] -> [b]
list(map(divide_by_two, arr))

[0.5, 1.0, 1.5, 2.0, 2.5]

In [13]:
list(filter(lambda x: x % 2 == 0, arr))

[2, 4]

# Map and Bind

```haskell
map :: (a -> b) -> [a] -> [b]
bind :: m a -> (a -> m b) -> m b
```


# Composition with `Maybe.bind`

`>>` operator is a shorthand for `bind` function

```python
def __rshift__(self, f: Callable[[T], 'Maybe[S]']) -> 'Maybe[S]':
    # a >> func = a.bind(func)
    return self.bind(f)
```


In [14]:
(
    divided_by_two(1024)
    >> divided_by_two
    >> divided_by_two
    >> divided_by_two
    >> divided_by_two
    >> divided_by_two
)

Just(16.0)

In [15]:
(
    divided_by_two(1024)
    >> divided_by_two
    >> divided_by_zero
    >> divided_by_two
    >> divided_by_two
    >> divided_by_two
)

Nothing